# Evolutionary Computation - Assignment 6: Multiple start local search (MSLS) and iterated local search (ILS)
Bartosz Stachowiak 148259<br>
Andrzej Kajdasz 148273

## 1. Problem Statement

There are columns of integers representing nodes. Each row corresponds to a node and contains its x and y coordinates in a plane, as well as a cost associated with the node. There were 4 such data sets each consisting of 200 rows (each representing a single node).

Problem to solve is to choose precisely 50% of the nodes (rounding up if there is an odd number of nodes) and create a Hamiltonian cycle (a closed path) using this subset of nodes. The goal is to minimize the combined total length of the path and the total cost of the selected nodes.

To calculate the distances between nodes, the Euclidean distance formula was used and then round the results to the nearest integer. As suggested, the distances between the nodes were calculated after loading the data and placed in a matrix, so that during the subsequent evaluation of the problem, it was only necessary to read these values which reduced the cost of the operation of the algorithm.

To solve the problem the multiple start local search and iterated local search were used.

## 2. Pseudocode of all implemented algorithms

### 2.1 Multiple start local search

```
def multiple_start_local_search(local_search, number_of_starts, nodes, distances):
    best_solution = None
    best_score = +inf

    for i in range(0, number_of_starts):
        solution = random_solution()
        solution = local_search.improve(solution, nodes, distances)
        score = evaluate(solution, nodes, distances)
        if score < best_score:
            best_score = score
            best_solution = solution
    return best_solution
```

### 2.2 Iterated local search

```
def perturb(solution, nodes, perturbation_size):
    for _ in range(0, perturbation_size):
        if random() < 0.5:
            solution = swapEdge(solution, randint(0, len(solution) - 1), randint(0, len(solution) - 1))
        else:
            nodes_outside_solution = [node for node in range(0, len(nodes)) if node not in solution]
            solution = replaceNode(solution, randint(0, len(solution) - 1), randint(0, len(solution) - 1))
    return solution


def iterated_local_search(local_search, max_time, perturbation_size, nodes, distances):
    start_time = time.time()
    best_solution = None
    best_score = +inf

    solution = local_search.improve(random_solution(), nodes, distances)
    while current_time() - start_time < max_time:
        peturbed_solution = perturb(solution, nodes, perturbation_size)
        peturbed_solution = local_search.improve(peturbed_solution, nodes, distances)
        score = evaluate(peturbed_solution, nodes, distances)
        if score < best_score:
            best_score = score
            best_solution = solution
        if score < evaluate(solution, nodes, distances):
            solution = peturbed_solution
    return best_solution
```

## 3. Results of the computational experiments

In [ ]:
import json
import pathlib
import itertools

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
from common import *

In [ ]:
DATA_FOLDER = '../data/'
OLD_RESULTS_FOLDER = f'{DATA_FOLDER}old_results/'
RESULT_FOLDER = f'{DATA_FOLDER}results/'
INSTANCE_FOLDER = f'{DATA_FOLDER}tsp_instances/'

SOLVERS = {
    "lsm-r" : "Steepest Multi Start LS",
    "lsi-3-r" : "Iterated LS (Perturbation size 3)",
    "lsi-5-r" : "Iterated LS (Perturbation size 5)",
    "lsi-10-r" : "Iterated LS (Perturbation size 10)",
    "lsi-20-r" : "Iterated LS (Perturbation size 20)",
    "lsi-30-r" : "Iterated LS (Perturbation size 30)",
    "lsi-50-r" : "Iterated LS (Perturbation size 50)",
}

OLD_SOLVERS = {
    "lsp-r" : "Steepest LS (delta reusing)",
}
SOLVERS = {**OLD_SOLVERS, **SOLVERS}
SOLVERS_TO_PLOT = SOLVERS.copy()
NUM_NODES = 200

instance_files = [path for path in pathlib.Path(INSTANCE_FOLDER).iterdir() if path.is_file()]
instance_names = [path.name[:4] for path in instance_files]
p_sizes = [3, 5, 10, 20, 30, 50]

In [ ]:
instances_data = {
    name: read_instance(f'{INSTANCE_FOLDER}{name}.csv')
    for name in instance_names
}

In [ ]:
instances_solvers_pairs = itertools.product(instances_data.keys(), SOLVERS.keys())

all_results = {}
all_costs = {}
all_times = {}
all_stats = {}

for instance, solver in instances_solvers_pairs:
    all_results[instance] = all_results.get(instance, {})
    all_costs[instance] = all_costs.get(instance, {})
    all_times[instance] = all_times.get(instance, {})
    all_stats[instance] = all_stats.get(instance, {})
    costs = []
    times = []
    paring_results = []
    file_number = NUM_NODES if solver in OLD_SOLVERS else 20
    for idx in range(file_number):
        folder = OLD_RESULTS_FOLDER if solver in OLD_SOLVERS else RESULT_FOLDER
        solution, cost, time = read_solution(f'{folder}{instance}-{solver}-{idx}.txt')
        paring_results.append(solution)
        costs.append(cost)
        times.append(time)
    all_results[instance][solver] = np.array(paring_results)
    all_costs[instance][solver] = np.array(costs)
    all_stats[instance][solver] = {
        'mean': np.mean(costs),
        'std': np.std(costs),
        'min': np.min(costs),
        'max': np.max(costs),
    }
    all_times[instance][solver] = {
        'mean': np.mean(times),
        'std': np.std(times),
        'min': np.min(times),
        'max': np.max(times),
    }

In [ ]:
costs_df = pd.DataFrame(all_stats).T
time_df = pd.DataFrame(all_times).T
max_df = pd.DataFrame(all_stats).T
min_df = pd.DataFrame(all_stats).T
mean_time_df = pd.DataFrame(all_times).T

for column in SOLVERS.keys():
    costs_df[column] = costs_df[column].apply(lambda x: f'{x["mean"]:.0f} ({x["min"]:.0f} - {x["max"]:.0f})')
    time_df[column] = time_df[column].apply(lambda x: f'{x["mean"]/1000000:.2f} ({x["min"]/1000000:.2f} - {x["max"]/1000000:.2f})')
    max_df[column] = max_df[column].apply(lambda x: x['max'])
    min_df[column] = min_df[column].apply(lambda x: x['min'])
    mean_time_df[column] = mean_time_df[column].apply(lambda x: x['mean']/1000000)

for df in [costs_df, time_df, max_df, min_df, mean_time_df]:
    df.rename(columns=SOLVERS, inplace=True)
time_df = time_df.filter(items = SOLVERS_TO_PLOT.values())
mean_time_df = mean_time_df.filter(items  = SOLVERS_TO_PLOT.values())

### 3.1. Visualizations and statistics of cost for all dataset-algorithm pairs

In tabular form we present the Mean, Minimum and Maximum of the results of the algorithms for each dataset.

In [ ]:
print("Mean (min-max) of the costs:")

best_means = {
    instance: min(all_stats[instance][solver]['mean'] for solver in SOLVERS.keys())
    for instance in instance_names
}

def apply_style(v: str, best_val: float):
    num = v.split()[0]
    try:
        num = float(num)
    except ValueError:
        return ""
    if round(num) == round(best_val):
        return "font-weight: bold; color: red"
    return ""
    


costs_df.T.style.apply(lambda x: [
    apply_style(v, best_means[x.index[i]])
    for i, v in enumerate(x)
], axis = 1)

### 3.2 Mean number of iterations for each ILS-Instance pair

In [ ]:
with open(f'{DATA_FOLDER}iterations.json', 'r') as f:
    iterations = json.load(f)

iterations_matrix = {
    instance: {
        size: iterations[f"{instance}-{size}"]
        for size in p_sizes
    } for instance in instance_names
}
it_df = pd.DataFrame(iterations_matrix)
it_df.index = [f'Number of operations in perturbation = {x}' for x in it_df.index]
it_df

 ### 3.3. Visualizations and statistics of running times for all dataset-algorithm pairs

In [ ]:
print("Mean (min-max) of the time [s]:")

best_times = {
    instance: min(all_times[instance][solver]['mean'] for solver in SOLVERS.keys()) /1000000
    for instance in instance_names
}

def apply_style(v: str, best_val: float):
    num = v.split()[0]
    try:
        num = float(num)
    except ValueError:
        return ""
    if round(num) == round(best_val):
        return "font-weight: bold; color: red"
    return ""
    


time_df.T.style.apply(lambda x: [
    apply_style(v, best_times[x.index[i]])
    for i, v in enumerate(x)
], axis = 1)

In [ ]:
x_range = np.arange(len(SOLVERS_TO_PLOT))
bar_width = 0.8 / len(instances_data.keys())
mean_time_plot_df = mean_time_df.T.sort_values(by="TSPA", ascending=False).T
fig, ax = plt.subplots(figsize=(15, 8), sharey=True)
for idx, instance in enumerate(instances_data.keys()):
     ax.bar(
          x_range + idx * bar_width,
          height=mean_time_plot_df.loc[instance].values,
          width=bar_width,
          label=instance,
     )
ax.set_xticks(x_range + bar_width * (len(instances_data.keys()) - 1) / 2)
ax.set_xticklabels(mean_time_plot_df.columns, rotation=45, ha='right')
plt.title('Time per instance per solver')
plt.ylabel('Running Time [s]')
plt.legend()
plt.show()


In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(14, 5))

for instance in instances_data.keys():
    
    axs[0].plot(
        p_sizes,
        [all_stats[instance][f"lsi-{n}-r"]['mean'] for n in p_sizes],
        label=instance,
        marker='o', 
        linestyle='dashed'
    )
    axs[1].plot(
        p_sizes,
        [iterations_matrix[instance][n] for n in p_sizes],
        label=instance,
        marker='o', 
        linestyle='dashed'
    )

axs[0].set_title('Mean cost of Iterated LS per size of perturbations')
axs[0].set_xlabel('Number of perturbations')
axs[0].set_ylabel('Mean cost')

axs[1].set_title('Mean number of iterations of Iterated LS per size of perturbations')
axs[1].set_xlabel('Number of perturbations')
axs[1].set_ylabel('Number of iterations')

plt.legend()
plt.show()

## 4. Best solutions for all datasets and algorithms

To more easily compare the results, we present the best solutions for each dataset side by side.

The weight of each node is denoted both by its size and color. The bigger and brighter the node, the higher its weight.

### 4.1 New algortithms

In [ ]:
for solver_idx, solver in enumerate(SOLVERS.keys()):
    fig, axs = plt.subplots(1, 4, figsize=(20, 5))
    for idx, instance in enumerate(instances_data.keys()):
        best_instance_idx = np.argmin(all_costs[instance][solver])
        plot_solution_for_instance(instances_data[instance], all_results[instance][solver][best_instance_idx], axs[idx])
        axs[idx].set_title(f'{instance}: {all_costs[instance][solver][best_instance_idx]:.0f}')
    fig.suptitle(f'{SOLVERS[solver]}', fontsize=16, y=1.05)
plt.show()

### 4.2 Best solution for each instance from all algorithms

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(20, 5))
for idx, instance in enumerate(instances_data.keys()):
    best_cost =  np.inf
    for solver_idx, solver in enumerate(SOLVERS.keys()):
         if best_cost > np.min(all_costs[instance][solver]):
                best_cost = np.min(all_costs[instance][solver])
                best_result = all_results[instance][solver][np.argmin(all_costs[instance][solver])], 
                best_solver = solver
    best_instance_idx = np.argmin(all_costs[instance][best_solver])
    plot_solution_for_instance(instances_data[instance], all_results[instance][best_solver][best_instance_idx], axs[idx])
    axs[idx].set_title(f'{instance}: {all_costs[instance][best_solver][best_instance_idx]:.0f}')
    print(instance)
    print(f'\tSolver: {SOLVERS[best_solver]}, Total cost: {best_cost}')
    nodes = list(best_result[0])
    if 0 in best_result[0]:
        zero_index = np.where(best_result[0] == 0)[0][0]
        nodes = list(best_result[0][zero_index:])+list(best_result[0][:zero_index])
    print(f'\t Nodes: {nodes}\n')
plt.show()

## 5. Source Code

[GitHub](https://github.com/Tremirre/ECP)

## 6. Conclusions



Analyzing the results and visualizations, one can come to several conclusions about the algorithms used in the task:
- Both Steepest Multi Start Local Search and Iterated Local Search outperform all previously used algorithms, at the cost of much longer running time.
- The average Multi Start Local Search running time varies widely, ranging from 5 seconds for the TSPD to 10 seconds for the TSPA (taking into account the minimum and maximum times, this is a quadruple difference from almost 4 to over 16 seconds)
- As the size of the perturbation increases, the number of iterations in Iterated LS decreases, but it does not automatically translate into a better result - optimal perturbation size has been found as ca. 10-20 operations.
- Regardless of perturbation size, the Iterated LS yields better results than the Multi Start LS, but the difference is not significant (does not exceed 3% of MSLS cost)
